In [ ]:
### imports

# external modules
import os
import sys
import json
import numpy as np
from fnmatch import fnmatch
import importlib

# local modules
thisdir = os.getcwd()
topdir = os.path.abspath(os.path.join(thisdir, '../../'))
sys.path.append(topdir)
import tools.omsapi.get_oms_data as oms

In [ ]:
# get the omsapi instance

omsapi = oms.get_oms_api()

In [ ]:
# define run ranges

# references:
# - https://twiki.cern.ch/twiki/bin/view/CMS/PdmVRun3Analysis#2024_Era_definition

eradict = {
  'Run2024A-v1': (378142, 378970),
  'Run2024B-v1': (378971, 379411),
  'Run2024C-v1': (379412, 380252),
  'Run2024D-v1': (380253, 380947),
  'Run2024E-v1': (380948, 381383),
  'Run2024E-v2': (381384, 381943),
  'Run2024F-v1': (381944, 383779),
  'Run2024G-v1': (383780, 385813),
  'Run2024H-v1': (385814, 386408),
  'Run2024I-v1': (386409, 386797),
  'Run2024I-v2': (386798, 387121),
  'Run2024J-v1': (387203, 387721)
}

In [ ]:
# find out which attributes are available per lumisection

runnb = 378142 # dummy run, should not matter
ls_info = oms.get_oms_data( omsapi, 'lumisections', runnb )
available_attributes = list(oms.get_oms_response_attributes(ls_info))
print(available_attributes)

In [ ]:
# define attributes to retrieve

attributes = [
    'delivered_lumi_per_lumisection',
    'recorded_lumi_per_lumisection',
    'pileup',
    'physics_flag',
    'fill_number',
    'run_number',
    'lumisection_number',
]

# quick check
for attribute in attributes:
    if attribute not in available_attributes:
        print(f'WARNING: attribute {attribute} does not seem to be available.')

In [ ]:
# get the data from OMS

importlib.reload(oms)
info = {}
for era, runrange in eradict.items():
    print('Retrieving data for era {}'.format(era))
    ls_info = oms.get_oms_data_iterative(omsapi, 'lumisections', runrange, attributes)
    print('Found {} lumisections'.format(len(ls_info[list(ls_info.keys())[0]])))
    info[era] = ls_info

In [ ]:
# ad-hoc fix for observed None values in physics flag

for era in eradict.keys():
    thisinfo = info[era]['physics_flag']
    nnone = thisinfo.count(None)
    if nnone==0: continue
    print('Found {} None instances (out of {} total) for era {}'.format(nnone, len(thisinfo), era))
    info[era]['physics_flag'] = [el if el is not None else False for el in thisinfo]

In [ ]:
# save to json files

outputdir = 'omsdata'
if not os.path.exists(outputdir): os.makedirs(outputdir)
    
for era in eradict.keys():
    thisinfo = info[era]
    outputfile = os.path.join(outputdir, 'omsdata_{}.json'.format(era))
    with open(outputfile, 'w') as f:
        json.dump(thisinfo, f)
    print(f'Created file {outputfile}')

In [ ]:
# get available HLT paths and attributes

importlib.reload(oms)

# pick a dummy run, should not matter
# (although might actually matter, trigger menu changes over time!)
runnb = 378142
ls_info = oms.get_oms_data( omsapi, 'hltpathinfo', runnb=runnb, attributes=['path_name'] )
hltpaths = oms.get_oms_response_attribute(ls_info, 'path_name')
#hltpaths = [p for p in hltpaths if p.startswith('HLT_ZeroBias')]
print('Available triggers:')
for p in hltpaths: print('  {}'.format(p))

# pick a trigger from the list above
trigger = 'DQM_PixelReconstruction_v8'
path_filter = {'attribute_name':'path_name', 'value':trigger, 'operator':'EQ'}
per_lumi_arg = {'group[granularity]': 'lumisection'}
ls_info_raw = oms.get_oms_data(omsapi, 'hltpathrates', runnb, extraargs=per_lumi_arg, extrafilters=[path_filter])
print('Available attributes')
print(oms.get_oms_response_attributes(ls_info_raw))

In [ ]:
# get the HLT rate from OMS
# note: only works for filtering per run, not run ranges.
# todo: works in principle, but need to implement a filter 
#       to query only the runs that are present in the DQMIO data,
#       rather than all runs present in OMS.

trigger_patterns = [
    'HLT_Physics_v*',
    'HLT_ZerioBias_v*',
    'DQM_PixelReconstruction_v*'
]

attributes = [
    'rate',
    'first_lumisection_number',
    'run_number'
]

# note: the attribute 'first_lumisection_number' actually corresponds to the lumisection number
#       when the rates are retrieved per lumisection.

outputdir = 'omsdata'
if not os.path.exists(outputdir): os.makedirs(outputdir)

for era, runrange in eradict.items():
    print('Retrieving data for era {}'.format(era))
    info = {}
    # get a list of runs
    runs = oms.get_oms_data( omsapi, 'runs', runnb=runrange, attributes=['run_number'] )
    runs = oms.get_oms_response_attribute(runs, 'run_number')
    print('Looping over {} runs'.format(len(runs)))
    # loop over runs
    for run in runs:
        info[run] = {}
        # get the trigger names for this run
        hltinfo = oms.get_oms_data( omsapi, 'hltpathinfo', runnb=run, attributes=['path_name'], limit_entries=10000 )
        hltpaths = oms.get_oms_response_attribute(hltinfo, 'path_name')
        triggers = []
        for hltpath in hltpaths:
            keep = False
            for pattern in trigger_patterns:
                if fnmatch(hltpath, pattern): keep = True
            if keep: triggers.append(hltpath)
        print('Run {}: found following triggers: {}'.format(run, triggers))
        # loop over trigger names
        for trigger in triggers:
            path_filter = {'attribute_name':'path_name','value':trigger,'operator':'EQ'}
            per_lumi_arg = {'group[granularity]':'lumisection'}
            ls_info_raw = oms.get_oms_data( omsapi, 'hltpathrates', runnb=run, attributes=attributes,
                                extraargs=per_lumi_arg, extrafilters=[path_filter], limit_entries=10000 )
            rate = {attribute: oms.get_oms_response_attribute(ls_info_raw, attribute) for attribute in attributes}
            info[run][trigger] = rate
            
        # store the information for this run
        # note: keep in loop to avoid losing everything over a transient issue.
        outputfile = os.path.join(outputdir, 'hltrate_{}_{}.json'.format(era, run))
        with open(outputfile, 'w') as f:
            json.dump(info[run], f)